# CS549 Machine Learning
# Assignment 8: Convolutional Neural Network (Part 3) -- Use pre-trained ResNet model

**Author:** Yang Xu, Assistant Professor of Computer Science, San Diego State University

**Total points: 10**

In this assignment, you will implement a more powerful classification model by building on top of a pre-trained ResNet model, and test its performance on the same FashionMNST dataset.

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import torchvision
from torchvision import datasets
from torchvision.transforms import transforms

## Load data

Load the FashionMNIST dataset provided by PyTorch. You can also change the `download` param to `False`, and copy the "data" folder used in the previous assignment to the current folder. See <https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader> for more information.

We also need to preprocess the dataset using a customized transform function, because the ResNet implemented in torchvision take colored image as input, which has 3 channels (RGB). Thus, we repeat the single-channel grey scale  image 3 times to fit the torchvision model, using a `lambda` expression.

In [2]:
transform = transforms.Compose([transforms.ToTensor(),
                transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
            ])

training_data = datasets.FashionMNIST(
    root="data",
    train=True, # True
    download=True,
    transform=transform
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False, # False
    download=True,
    transform=transform
)

batch_size = 64

train_loader = DataLoader(training_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

## Examine data size

Now, you can examine the size of the training/test data, which is important for determining some of the parameters of your model

In [3]:
for i, (X, y) in enumerate(train_loader):
    if i > 0:
        break

print('X.shape: ', X.shape)
print('Y.shape: ', y.shape)

X.shape:  torch.Size([64, 3, 28, 28])
Y.shape:  torch.Size([64])


**Expected output**:

X.shape:  torch.Size([64, 3, 28, 28])
y.shape:  torch.Size([64])

**Note** that the number of channels becomes 3!

## Task 1. Build the extractor model using pre-trained ResNet model

We can directly download and use the pretrained ResNet model by calling the constructor `torchvision.models.resnet18()` (or other versions of ResNet), with argument `pretrained=True`.

Then we will use the components of the pretrained model as a **feature extractor**. The feature extractor is a separate class from the final classification model, because the best learning rates (and other hyper-parameters) for training the two models can be different.

First, let's implement the feature extractor.

**Points: 5**

*Hint*: Call `torchvision.models.resnet18()` in the `__init__()` function, and use copy the components in the pretrained model into the current one.

In [4]:
class ResNetFeatrueExtractor(nn.Module):
    def __init__(self):
        super(ResNetFeatrueExtractor, self).__init__()
        ### START YOUR CODE ###
        resnet18_model = torchvision.models.resnet18(pretrained=True) # Call torchvision.models.resnet18()

        self.conv1 = resnet18_model.conv1 # Complete all the lines below following the same naming pattern: self.NAME = resnet18_model.NAME
        self.bn1 = resnet18_model.bn1
        self.relu = resnet18_model.relu
        self.maxpool = resnet18_model.maxpool
        self.layer1 = resnet18_model.layer1
        self.layer2 = resnet18_model.layer2
        self.layer3 = resnet18_model.layer3
        self.layer4 = resnet18_model.layer4
        self.avgpool = resnet18_model.avgpool
        ### END YOUR CODE ###

    def forward(self, x):
        ### START YOUR CODE ###
        # Call all the components consecutively, i.e., conv1 -> bn1 -> relu -> maxpool -> ...
        x = self.avgpool(
            self.layer4(
                self.layer3(
                    self.layer2(
                        self.layer1(
                            self.maxpool(
                                self.relu(
                                    self.bn1(
                                        self.conv1(x)
                                    )
                                )
                            )
                        )
                    )
                )
            )
        ) # There can be multiple lines
        ### END YOUR CODE ###

        x = x.view(x.size(0), -1)

        return x

In [5]:
# Do not change the test code here
torch.manual_seed(0)
extractor = ResNetFeatrueExtractor()

input_data = torch.randn(64, 3, 28, 28)
output = extractor(input_data)

print('output.size():', output.size())

output.size(): torch.Size([64, 512])


**Expected output**

output.size(): torch.Size([64, 512])

---

## Task 2. Build the classifier model

**Points: 2**

The classifier model takes the output from the extractor, and feed it to a fully connected layer and then a softmax output.

In [6]:
class ResNetClassifier(nn.Module):
    def __init__(self):
        super(ResNetClassifier, self).__init__()
        ### START YOUR CODE ###
        self.fc = nn.Linear(in_features=512, out_features=10) # Specify the in_features and out_features correctly
        self.output = nn.LogSoftmax(dim=1) # Call nn.LogSoftmax()
        ### END YOUR CODE ###

    def forward(self, x):
        ### START YOUR CODE ###
        out = self.output(self.fc(x))
        ### END YOUR CODE ###
        return out

In [7]:
# Do not change the test code here
torch.manual_seed(0)
clasifier = ResNetClassifier()

input_data = torch.randn(64, 512)
output = clasifier(input_data)

print('output.size():', output.size())

output.size(): torch.Size([64, 10])


**Expected output**

output.size(): torch.Size([64, 10])

---

## Task 3. Implement the training and test loops

**Points: 3**

Implement the training and test loop functions. Note that
- Image data are input for `extractor`, whose output is the input for `classifier`.
- Two optimizers need to be used, one for `extractor`, and the other for `classifier`.

In [8]:
def train_loop(dataloader, extractor, classifier, loss_fn, optimizer_ext, optimizer_cls, verbose=True):
    for i, (X, y) in enumerate(dataloader):
        ### START YOUR CODE ###
        # Extract features using extractor
        features = extractor(X)

        # Feed the features to classifier
        pred = classifier(features)

        # Compute loss
        loss = loss_fn(pred, y)
        ### END YOUR CODE ###

        # Backpropagation
        ### START YOUR CODE ###
        # Clear the gradients for the TWO optimizers!
        optimizer_ext.zero_grad()
        optimizer_cls.zero_grad()

        # Call backward()
        loss.backward() # backward()

        # Call step() for the TWO optimizers!
        optimizer_ext.step()
        optimizer_cls.step()
        ### END YOUR CODE ###

        if verbose and i % 10 == 0:
            loss = loss.item()
            current_step = i * len(X)
            print(f"loss: {loss:>7f}  [{current_step:>5d}/{len(dataloader.dataset):>5d}]")

In [9]:
@torch.no_grad()
def test_loop(dataloader, extractor, classifier, loss_fn):
    test_loss, correct = 0, 0

    for X, y in dataloader:
        ### START YOUR CODE ###
        # The code for computing loss is similar to train_loop()
        features = extractor(X)
        pred = classifier(features)
        loss = loss_fn(pred, y)
        test_loss += loss.item()
        correct += torch.mean((torch.argmax(pred, dim=1) == y).float()) # Add the number of correct prediction in the current batch to `correct`
        ### END YOUR CODE ###

    test_loss /= len(dataloader)
    test_acc = correct / len(dataloader.dataset)

    print(f"Test Error: \n Accuracy: {(100*test_acc):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Finally, run the training and test loop functions. Training a ResNet model is very slow on CPU, so we just try 1 epoch here, and be patient while you are waiting for the output.

In [10]:
extractor = ResNetFeatrueExtractor()
classifier = ResNetClassifier()
learning_rate = 1e-3

### START YOUR CODE ###
loss_fn = nn.NLLLoss() # Specify the loss function correctly
optimizer_ext = torch.optim.Adam(params=extractor.parameters(), lr=learning_rate) # Use Adam optimizer on extractor
optimizer_cls = torch.optim.Adam(params=classifier.parameters(), lr=learning_rate) # Use Adam optimizer on classifier
### END YOUR CODE ###

epochs = 1
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, extractor, classifier, loss_fn, optimizer_ext, optimizer_cls, verbose=True) # Use verbose=False, if you want to see less information
    test_loop(test_loader, extractor, classifier, loss_fn)

print("Done!")

Epoch 1
-------------------------------
loss: 2.405839  [    0/60000]
loss: 0.730246  [  640/60000]
loss: 0.860524  [ 1280/60000]
loss: 0.670298  [ 1920/60000]
loss: 0.623385  [ 2560/60000]
loss: 0.654261  [ 3200/60000]
loss: 0.539835  [ 3840/60000]
loss: 0.577238  [ 4480/60000]
loss: 0.524115  [ 5120/60000]
loss: 0.397807  [ 5760/60000]
loss: 0.490525  [ 6400/60000]
loss: 0.423573  [ 7040/60000]
loss: 0.503942  [ 7680/60000]
loss: 0.500117  [ 8320/60000]
loss: 0.559954  [ 8960/60000]
loss: 0.498742  [ 9600/60000]
loss: 0.391623  [10240/60000]
loss: 0.372260  [10880/60000]
loss: 0.581777  [11520/60000]
loss: 0.440633  [12160/60000]
loss: 0.303565  [12800/60000]
loss: 0.359001  [13440/60000]
loss: 0.474043  [14080/60000]
loss: 0.452061  [14720/60000]
loss: 0.461027  [15360/60000]
loss: 0.570573  [16000/60000]
loss: 0.486111  [16640/60000]
loss: 0.355791  [17280/60000]
loss: 0.560113  [17920/60000]
loss: 0.326118  [18560/60000]
loss: 0.387523  [19200/60000]
loss: 0.541789  [19840/60000]


Although it is just 1 epoch, you can find that the performance is pretty nice. It shows that using a pretrained model is a good strategy!